In [3]:
# QR Solution
using LinearAlgebra
function consistentLS(A,b)
    """
    Solves a consistent linear system given
    the coefficient matrix A and the constant
    vector b. Assumes A is consistent.
    """
    n, m = size(A)
    F = qr(A,Val(true))
    d = F.Q'*b
    c = F.R\d[1:m]
    return F.P*c
end

function underLSQR(A,b; ϵ = 1e-14)
    """
    Solves an underdetermined linear system given
    the coefficient matrix A and the constant
    vector b. Returns the least norm solution.
    Uses the QR decomposition.
    """
    n, m = size(A)
    s = min(n,m)
    F = qr(A, Val(true))
    
    #Compute rank approximation r
    Rtrm = F.R[1:s,1:s]
    r = maximum(findall(abs.(diag(Rtrm)) .>= ϵ))
    l = m - r
    
    #Generate R and S
    R, S = F.R[1:r,1:r], F.R[1:r,r+1:end]
    d, P = R\(F.Q'*b)[1:r], R\S
    z2 = consistentLS(P'*P + Matrix(I,l,l), P'*d)
    z1 = d - P*z2
    return F.P*vcat(z1,z2)
end

# SVD Solution
function underLSSVD(A,b; ϵ = 1e-14)
    """
    Solves a (possibly) underdetermined linear
    systems given the coefficient matrix A and
    constant vector b using SVD of A. Returns
    the least norm solution.
    """
    n, m = size(A)
    U, Σ, V = svd(A, full=true)
    c = U'*b

    #Determine c₁ and solution
    z = n < m ? vcat(c./Σ,zeros(m-n)) : c./Σ #Σ is vector
    return V*z
end;

In [4]:
# Example of Solving Linear System

n, m = 10, 20
printstyled(
"EXAMPLE 1: SVD vs QR\n", color=:red)

println(
"Computing minimum-norm LS\n")

A = rand(n,m)
x = rand(m)
b = A*x

x₁ = underLSSVD(A,b)
x₂ = underLSQR(A,b)

println(
"Residual for SVD:
    $(norm(A*x₁ - b))")
println(
"Residual for QR:
    $(norm(A*x₂ - b))")
println(
"Error SVD and QR:
    $(norm(x₁-x₂))")


EXAMPLE 1: SVD vs QR
Computing minimum-norm LS

Residual for SVD:
    2.188238034285014e-14
Residual for QR:
    4.699798436761765e-15
Error SVD and QR:
    5.2738515228306065e-15


In [6]:
# Example Norm Calculation
n, m = 10, 20
printstyled(
"EXAMPLE 2: 2-Norm\n", color=:red)

println(
"Computing Matrix 2-Norm\n")

A = rand(n,m)

println("Julia's 2-Norm of A:
    $(norm(A,2))\n")

println("Max Singular Value of A:
    $(maximum(svdvals(A)))")



EXAMPLE 2: 2-Norm
Computing Matrix 2-Norm

Julia's 2-Norm of A:
    8.529360972041529

Max Singular Value of A:
    7.683779428243258


In [10]:
#Approximate Rank K Calculation

using Distributions

function inexactRankK(A,k,c,p)
    """
    Inexact rank k approximation to a matrix A.
    c is an integer betweek k and m.
    p is an m-tuple probabilities summing to 1.
    """
    dist = Categorical(p)
    cols_ind = rand(dist,c)

    #Divide each col by sqrt(c*probability)
    C = A[:,cols_ind]./sqrt.(c*p[cols_ind]')

    #Compute eigenvalue decomposition
    D, V = eigen(C'*C)

    #Compute top k singular values
    S = sqrt.(D[end-k+1:end])

    #Take top k right singular vec of C
    Z = V[:,end-k+1:end]

    #Compute left singular vectors
    H = C*(Z./S')
    return H, S
end

#
# Examples
#
#Generate Random Matrices

function generateLowRank(n,m)
    m <= n &&
    error("First argument ($n) must be smaller than second ($m)")
    r = rand(1:n)
    U, _ = qr(rand(n,n))
    V, _ = qr(rand(m,m))
    Σ = hcat(diagm(0 => vcat(100*rand(r),zeros(n-r))),zeros(n,m-n))
    return U*Σ*V', r
end

#Example 3
n, m = 30,500
A, r = generateLowRank(n,m)
printstyled(
"EXAMPLE 3: $n by $m matrix with rank $r\n\n", color=:red)

k, c, p = max(r-10,1), 30, ones(m)/m
H, S = inexactRankK(A,k,c,p)
err = norm(A - H*H'*A,2)^2
println("k=$k and c=$c, error: $err\n")

k, c, p = max(r-10,1), 60, ones(m)/m
H, S = inexactRankK(A,k,c,p)
err = norm(A - H*H'*A,2)^2
println("k=$k and c=$c, error: $err\n")

k, c, p = max(r-10,1), 120, ones(m)/m
H, S = inexactRankK(A,k,c,p)
err = norm(A - H*H'*A,2)^2
println("k=$k and c=$c, error: $err\n")

k, c, p = r, 30, ones(m)/m
H, S = inexactRankK(A,k,c,p)
err = norm(A - H*H'*A,2)^2
println("k=$k and c=$c, error: $err\n")

k, c, p = r, 60, ones(m)/m
H, S = inexactRankK(A,k,c,p)
err = norm(A - H*H'*A,2)^2
println("k=$k and c=$c, error: $err\n")

k, c, p = r, 120, ones(m)/m
H, S = inexactRankK(A,k,c,p)
err = norm(A - H*H'*A,2)^2
println("k=$k and c=$c, error: $err\n")


EXAMPLE 3: 30 by 500 matrix with rank 8

k=1 and c=30, error: 30272.292745712122

k=1 and c=60, error: 29176.966148066796

k=1 and c=120, error: 28678.181889900246

k=8 and c=30, error: 1.3251853240520431e-25

k=8 and c=60, error: 1.7761526530130944e-25

k=8 and c=120, error: 9.225877613680205e-26

